# WIP
currently porting original synapse code I developed with abfss over to this format

In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType}
import spark.implicits._

val lookupPath = ("")
val df = spark.read.json("/securitydatasets/datasets/atomic/windows/credential_access/host/empire_mimikatz_logonpasswords_2020-08-07103224.json")
df.write.format("delta").mode("overwrite").save(lookupPath)
df.count()

(console):1: unclosed string literal
val lookupPath = (\"abfss://prerecordedcl@securitysets.dfs.core.windows.net/securitydatasets/scratch\")
                   ^
(console):1: unclosed string literal
val df = spark.read.json(\"abfss://prerecordedcl@securitysets.dfs.core.windows.net/securitydatasets/datasets/atomic/windows/credential_access/host/empire_mimikatz_logonpasswords_2020-08-07103224.json\")
                          ^
(console):1: unclosed string literal
df.write.format(\"delta\").mode(\"overwrite\").save(lookupPath)
                 ^

: 

In [ ]:
df.select("@timestamp","Hostname","SubjectUserName","ProcessName","ObjectName","AccessMask","EventID")
.filter(
    ('EventID === 4663 || 'EventID === 4656)
    && 'ObjectName.like("%lsass.exe")
    && !'SubjectUserName.like("%$")).show()
)

(console):1:123 expected (Semis | &"}" | end-of-input)
"df.select(\"@timestamp\",\"Hostname\",\"SubjectUserName\",\"ProcessName\",\"ObjectName\",\"AccessMask\",\"EventID\")\r\n",
                                                                                                                          ^

: 

In [ ]:
val processAccessDf = df.select("@timestamp","Hostname","SourceImage","TargetImage","GrantedAccess","SourceProcessGUID","CallTrace")
.filter(
    'Channel === "Microsoft-Windows-Sysmon/Operational"
    && 'EventID === 10
    && 'TargetImage.like("%lsass.exe")
    && 'CallTrace.like("%UNKNOWN%"))"

(console):1:153 expected (Semis | &"}" | end-of-input)
"val processAccessDf = df.select(\"@timestamp\",\"Hostname\",\"SourceImage\",\"TargetImage\",\"GrantedAccess\",\"SourceProcessGUID\",\"CallTrace\")\r\n",
                                                                                                                                                        ^

: 

In [ ]:
val imageLoadDf = df.select("ProcessGuid","Image","ImageLoaded").filter(
    'Channel === "Microsoft-Windows-Sysmon/Operational"
    && 'EventID === 7
    && ( 
        'ImageLoaded.like("%samlib.dll") || 'ImageLoaded.like("%vaultcli.dll") || 'ImageLoaded.like("%hid.dll") || 'ImageLoaded.like("%winscard.dll") || 'ImageLoaded.like("%cryptdll.dll")
    )
)

(console):1:85 expected (Semis | &"}" | end-of-input)
"val imageLoadDf = df.select(\"ProcessGuid\",\"Image\",\"ImageLoaded\").filter(\r\n",
                                                                                    ^

: 

In [1]:
processAccessDf.join(imageLoadDf,processAccessDf("SourceProcessGUID") === imageLoadDf("ProcessGuid"),"inner").show()

res0: String = "processAccessDf.join(imageLoadDf,processAccessDf(\"SourceProcessGUID\") === imageLoadDf(\"ProcessGuid\"),\"inner\").show()"